In [12]:
# Импорт необходимых модулей и  библиотек
import pandas as pd
import numpy as np 
import sqlite3
import scipy.stats as stats
from scipy.stats import norm
# Функция для вычисления 95% доверительного интервала для разницы пропорций
from statsmodels.stats.proportion import proportion_confint, proportions_ztest

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, classification_report, precision_score, accuracy_score, recall_score
from sklearn.linear_model import LogisticRegression 
import warnings 
warnings.filterwarnings("ignore")
%matplotlib inline



In [13]:
# Прочитаем нашу БД SQLite
conn = sqlite3.connect('data/testcase.db')

# Прочитаем таблицы из базы данных для ознакомления
table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables = {name: pd.read_sql_query(f"SELECT * FROM {name}", conn) for name in table_names['name']}

# Закроем соединение
conn.close()

# Выводим информацию о таблицах
for name, table in tables.items():
    display(f"Таблица {name}", table.head())
    

'Таблица source_comparison'

,Install_Dates,source_type,Country,installs
0,2020-05-18,Paid,PY,4.0
1,2020-04-14,Paid,FR,35.0
2,2020-04-30,Paid,JP,25.0
3,2020-03-20,Paid,DE,11.0
4,2020-04-30,Paid,IT,8.0


'Таблица costs'

,Install_Dates,campaign_id,Country,installs,spends
0,2020-04-14,90570,CZ,35.0,19.79
1,2020-04-28,90619,AT,5.0,4.99
2,2020-05-10,794235,PK,79.0,0.40
3,2020-05-07,91872,FR,12.0,9.56
4,2020-04-03,19115,JP,6.0,3.39


'Таблица revenue'

,Install_Dates,campaign_id,Country,1d_LTV,3d_LTV,7d_LTV,14d_LTV,30d_LTV,60d_LTV
0,2020-04-14,90570,CZ,7.721194,8.104811,10.209480,11.845200,15.349594,15.768420
1,2020-03-30,90262,CA,1.424363,1.724738,1.724738,1.724738,1.724738,1.999413
2,2020-04-02,19115,PL,26.912006,32.943050,66.787497,70.071922,71.644023,72.057026
3,2020-04-15,788948,MD,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2020-03-26,158583,DE,0.349793,0.349793,0.349793,0.349793,0.349793,0.349793


### # 1 Успешность прототипа

In [14]:

# Классический подход
prob_classical = 0 / 200

# Байесовский подход
alpha_prior = 1
beta_prior = 1

# Наблюдения
successful_prototypes = 0
total_prototypes = 200 # Первые 200 прототипов 

# Параметры постериорного распределения
alpha_posterior = alpha_prior + successful_prototypes
beta_posterior = beta_prior + total_prototypes - successful_prototypes

# Вероятность успеха по Байесовскому подходу
prob_bayesian = alpha_posterior / (alpha_posterior + beta_posterior)

print(f"Вероятность успеха по классическому подходу: {prob_classical}")
print(f"Вероятность успеха по Байесовскому подходу: {prob_bayesian:.5f}")


Вероятность успеха по классическому подходу: 0.0
Вероятность успеха по Байесовскому подходу: 0.00495


#### Вывод: 
#### Классический подход: Вероятность успеха 0 (основана на нулевом количестве успешных прототипов из 200).
#### Байесовский подход: Вероятность успеха ≈ 0.00495 (учитывает априорные предположения и обновляется данными).

#### Для корректного байесовского анализа мы используем данные о первых 200 прототипах, а 201-й прототип — это тот, для которого мы прогнозируем вероятность успеха.

### # 2 Сравнение групп платящих игроков

### 1. Ответим на первый вопрос - Оптимальный дизайн эксперимента
### Описание эксперимента:

### Цель эксперимента:

* Проверить гипотезу о том, что обновление увеличит процент платящих игроков с 10% до 11%.

### Тип эксперимента:

* Рандомизированное контролируемое испытание (RCT (Randomized Controlled Trials)): Подразумевает случайное распределение участников на контрольную и экспериментальную группы, что позволяет минимизировать систематические ошибки и обеспечить объективность результатов.

### Группы:

* Контрольная группа: Получает текущую версию игры.
* Экспериментальная группа: Получает обновленную версию игры.
Метрики:

* Основная метрика: Процент платящих игроков (конверсия).
* Дополнительные метрики: Средний доход на платящего игрока (ARPU), чтобы понять не только процент, но и поведение платящих игроков.
### Размер выборки и статистическая мощность:

Используем формулу для сравнения пропорций, чтобы определить необходимое количество участников для статистически значимого результата. Это нам  поможет установить, сколько новых игроков нужно для наблюдения различий между группами.

### 2. Расчет длительности эксперимента
#### Добавим пояснение: 
* Поток новых игроков: Примерно 100 новых игроков в день.
* Размер выборки: Используем формулу для расчета, исходя из увеличения процента платящих с 10% до 11%.
* Обращаем внимание на ошибки I  и II рода 
- Ошибка 1 рода в том что мы отклонили H0 в то время когда она верна 
- Ошибка 2 рода в том что мы приняли H0 в то время когда она не верна

In [15]:
# Параметры
alpha = 0.05  # уровень значимости
beta = 0.2  # вероятность ошибки второго рода 
p1 = 0.10  # текущий процент платящих
p2 = 0.11  # ожидаемый процент платящих
p = (p1 + p2) / 2  # средняя пропорция

# Z-значения
Z_alpha2 = 1.96  # Z-значение для 95% CI(доверительный интервал)
Z_beta = 0.84   # Z-значение для мощности 80%

# Расчет размера выборки
n = ((Z_alpha2 * np.sqrt(2 * p * (1 - p)) + Z_beta * np.sqrt(p1 * (1 - p1) + p2 * (1 - p2))) ** 2) / (p2 - p1) ** 2
n = np.ceil(n)  # округление до ближайшего целого

# Продолжительность
players_per_day = 100
duration_days = n / players_per_day

n, duration_days  # возвращает размер выборки и необходимую продолжительность эксперимента в днях

(14735.0, 147.35)

#### Выводы: 
#### Таким образом, для достижения статистической значимости с α = 0.05 и мощностью β = 0.80, вам потребуется около 14735 игроков на группу, что при 100 новых игроках в день составит около 147 дней.
#### Формула учитывает изменение пропорций и использует Z-значения для определения необходимой выборки.
#### Вычисленная длительность эксперимента дает время, необходимое для набора достаточного количества новых игроков.

### 3. Генерация контрольного датасета  с около 10% плательщиков (контроль) и рассчитайте 95% HDI / CI.

In [16]:
# Генерация данных для контрольной группы
np.random.seed(0)
total_players = 500 # общее количество игроков это n
conversion_rate = 0.10 # текущий процент платящих это p
data = np.random.binomial(1, conversion_rate, total_players) # генерация данных

# Создание DataFrame
control_df = pd.DataFrame({'player_id': np.arange(total_players), 'paid': data})

# Вычисление 95% доверительного интервала для пропорции платящих
paid_count = control_df['paid'].sum()
ci_low, ci_high = proportion_confint(paid_count, total_players, alpha=0.05, method='wilson')

print("Контрольная группа:")
print("Конверсия:", control_df['paid'].mean())
print("95% CI:", (ci_low, ci_high))

Контрольная группа:
Конверсия: 0.114
95% CI: (0.08903690099400285, 0.1448490898423155)


#### Выводы: 

#### Нашей целью являлось создание датасета, который отражает 10% платящих игроков.

#### Используемая функция: Случайная генерация на основе биномиального распределения, характеризуется 2 параметрами: успеха в каждом эксперименте (обычно обозначается как p) и количеством экспериментов (обычно обозначается как n).
#### Биномиальное распределение используется для генерации данных, где вероятность успеха соответствует текущему проценту платящих (10%).
#### Доверительный интервал вычисляется с помощью метода Уилсона для вероятности успехов бинарных экспериментов (где результат может быть "успех" или "неудача"), обеспечивая более точную оценку для пропорций в малых выборках.

### 4. Генерация экспериментальных групп
#### Добавим пояснение:
* Создание экспериментальных датасетов:
* Варианты: Платящих 9% (ухудшение), 10% (без эффекта), 11% (улучшение).
* Генерация: Аналогично контрольной группе, но с различными коэффициентами конверсии.

In [17]:
# Варианты конверсий для эксперимента
rates = [0.09, 0.10, 0.11] # проценты платящих для разных сценариев
experiment_dfs = {} # словарь для хранения экспериментальных датасетов

for rate in rates:
    data = np.random.binomial(1, rate, total_players)  # генерация данных
    experiment_dfs[rate] = pd.DataFrame({'player_id': np.arange(total_players), 'paid': data}) # создание DataFrame

experiment_results = {}
for rate, df in experiment_dfs.items():
    paid_count = df['paid'].sum()
    ci_low, ci_high = proportion_confint(paid_count, total_players, alpha=0.05, method='wilson')
    experiment_results[rate] = {'rate': rate, 'ci': (ci_low, ci_high)}

print("Экспериментальные группы:")
for rate, result in experiment_results.items():
    print(f"Конверсия: {rate}, 95% CI: {result['ci']}")

Экспериментальные группы:
Конверсия: 0.09, 95% CI: (0.0714264085922213, 0.12276455586255913)
Конверсия: 0.1, 95% CI: (0.10691500187152081, 0.16666601534633546)
Конверсия: 0.11, 95% CI: (0.08903690099400285, 0.1448490898423155)


#### Вывод: Использовали частотный метод для сравнения конверсий и вычисления доверительных интервалов.
#### Результаты анализа показывают отсутствие значимого эффекта от обновления игры. Этот вывод основан на высоком p-значении и доверительном интервале, охватывающем 0.

### 5. Решение задачи частотным и байесовским методами

In [18]:
# Частотный подход
frequentist_results = {}

for rate, df in experiment_dfs.items():
    # Количество платящих игроков в экспериментальной группе
    experiment_success = df['paid'].sum()
    # Общее количество игроков в экспериментальной группе
    experiment_nobs = len(df)
    
    # Создание списков для количества успехов и размеров выборок
    count = [control_df['paid'].sum(), experiment_success]
    nobs = [len(control_df), experiment_nobs]
    
    # Выполнение Z-теста для разницы пропорций
    stat, p_value = proportions_ztest(count, nobs)
    
    # Сохранение результатов Z-теста
    frequentist_results[rate] = {
        'z_stat': stat,
        'p_value': p_value
    }

# Вывод результатов частотного подхода
print("Результаты частотного подхода:")
for rate, result in frequentist_results.items():
    print(f"Конверсия: {rate}, z-статистика: {result['z_stat']}, p-значение: {result['p_value']}")


# Определение функции для Байесовского обновления
def bayesian_update(control_success, control_size, experiment_success, experiment_size, alpha_prior=1, beta_prior=1):
    # Обновленные параметры Beta распределения для контрольной группы
    alpha_control_post = alpha_prior + control_success
    beta_control_post = beta_prior + (control_size - control_success)
    
    # Обновленные параметры Beta распределения для экспериментальной группы
    alpha_experiment_post = alpha_prior + experiment_success
    beta_experiment_post = beta_prior + (experiment_size - experiment_success)

    # Генерация выборок из обновленных Beta распределений
    samples_control = np.random.beta(alpha_control_post, beta_control_post, size=10000)
    samples_experiment = np.random.beta(alpha_experiment_post, beta_experiment_post, size=10000)

    # Вычисление разницы выборок
    samples_diff = samples_experiment - samples_control
    
    # Определение 95% HDI (Highest Density Interval)
    hdi_low, hdi_high = np.percentile(samples_diff, [2.5, 97.5])

    return samples_diff, hdi_low, hdi_high

# Байесовский подход
control_success = control_df['paid'].sum()
control_size = len(control_df)

bayesian_results = {}

for rate, df in experiment_dfs.items():
    # Количество платящих игроков в экспериментальной группе
    experiment_success = df['paid'].sum()
    # Общее количество игроков в экспериментальной группе
    experiment_size = len(df)
    
    # Выполнение Байесовского обновления
    samples_diff, hdi_low, hdi_high = bayesian_update(control_success, control_size, experiment_success, experiment_size)
    
    # Сохранение результатов Байесовского подхода
    bayesian_results[rate] = {
        'hdi_low': hdi_low,
        'hdi_high': hdi_high
    }

# Вывод результатов Байесовского подхода
print("Результаты байесовского подхода:")
for rate, result in bayesian_results.items():
    print(f"Конверсия: {rate}, 95% HDI: ({result['hdi_low']}, {result['hdi_high']})")

Результаты частотного подхода:
Конверсия: 0.09, z-статистика: 1.035927412705931, p-значение: 0.3002360077234014
Конверсия: 0.1, z-статистика: -0.9594826022626876, p-значение: 0.3373156800288286
Конверсия: 0.11, z-статистика: 0.0, p-значение: 1.0
Результаты байесовского подхода:
Конверсия: 0.09, 95% HDI: (-0.05787282688062041, 0.01832103713546915)
Конверсия: 0.1, 95% HDI: (-0.021180137285258693, 0.06152134416896951)
Конверсия: 0.11, 95% HDI: (-0.038704907784640745, 0.03984072600403806)


#### Вывод: 
* Z-тест сравнивает доли платящих между контрольной и экспериментальной группами, выводя статистику теста и p-значение.
* Байесовская модель: Здесь используется простая бета-распределение для априорных и апостериорных распределений, чтобы смоделировать вероятности.

* Сэмплирование: Из распределений помогает определить вероятностное распределение разницы в конверсии.

* Используемые методы позволяют сравнить процент платящих игроков до и после обновления игры, используя как частотные, так и простые байесовские методы для оценки результатов эксперимента. Каждый шаг подробно описан с комментариями, объясняющими выбор и выполнение методов, что обеспечивает понимание и репликацию результатов.

* Результаты показывают, что ни один из экспериментов (с разными конверсиями) не дает статистически значимых различий по сравнению с контрольной группой как в частотном, так и в байесовском анализах. Это указывает на то, что обновление, скорее всего, не увеличивает процент платящих игроков с 10% до 11%, что соответствует исходной гипотезе геймдизайнеров.

### # 3 Предсказание оттока

####  Для начала разделим всё на шаги чтобы было проще решать задачу: 

#### Шаги:

#### Шаг 1: Подготовка данных
Объединение данных:

Таблицу source_comparison можно использовать для анализа источников, откуда пришли игроки.
Таблицу costs можно использовать для анализа затрат на привлечение пользователей.
Таблицу revenue можно использовать для анализа дохода от пользователей.
Объединяем данные из всех таблиц по соответствующим полям.

Создание признаков:
Для каждого игрока извлекаем шаг, на котором он перестал продвигаться по туториалу (если это возможно).
Создадим признаки на основе последовательности действий игрока в туториале.

Маркировка данных:
Обозначим строки данных как положительные примеры (игрок ушел) или отрицательные (игрок завершил туториал).
#### Шаг 2: Построение модели

Выбор модели:
Для предсказания оттока можно использовать модели классификации, такие как логистическая регрессия, случайный лес, градиентный бустинг и тд. Мы используем эти 3 модели.

Обучение модели:
Обучаем модель на исторических данных, используя признаки, которые мы создали, и метки оттока.

Валидация модели:
Используем методы кросс-валидации, чтобы проверить стабильность и точность модели.

#### Шаг 3: Оценка модели

Метрики оценки:
Используем метрики, такие как Roc-Auc, F1-score, точность (precision), полнота (recall) для оценки качества модели.

Оценка с бизнес-стороны:
Оцениваем, насколько хорошо модель предсказывает отток игроков и как это влияет на ключевые бизнес-метрики, такие как удержание пользователей и доход.

#### Шаг 4: Дизайн эксперимента

Контрольная и тестовая группы:
Разделим игроков на контрольную и тестовую группы. В контрольной группе игроки будут проходить стандартный туториал, а в тестовой — альтернативный.

Запуск эксперимента:
Применяем предсказания модели к тестовой группе и предложим альтернативный туториал на шагах, где модель предсказывает высокий риск оттока.

#### Шаг 5: Заключение

Оценка результатов:
Сравниваем показатели оттока и прохождения туториала между контрольной и тестовой группами, чтобы проверить гипотезу геймдизайнеров.

In [19]:
# Выводим информацию о таблицах
for name, table in tables.items():
    display(f"Таблица {name}", table.head())
    

'Таблица source_comparison'

,Install_Dates,source_type,Country,installs
0,2020-05-18,Paid,PY,4.0
1,2020-04-14,Paid,FR,35.0
2,2020-04-30,Paid,JP,25.0
3,2020-03-20,Paid,DE,11.0
4,2020-04-30,Paid,IT,8.0


'Таблица costs'

,Install_Dates,campaign_id,Country,installs,spends
0,2020-04-14,90570,CZ,35.0,19.79
1,2020-04-28,90619,AT,5.0,4.99
2,2020-05-10,794235,PK,79.0,0.40
3,2020-05-07,91872,FR,12.0,9.56
4,2020-04-03,19115,JP,6.0,3.39


'Таблица revenue'

,Install_Dates,campaign_id,Country,1d_LTV,3d_LTV,7d_LTV,14d_LTV,30d_LTV,60d_LTV
0,2020-04-14,90570,CZ,7.721194,8.104811,10.209480,11.845200,15.349594,15.768420
1,2020-03-30,90262,CA,1.424363,1.724738,1.724738,1.724738,1.724738,1.999413
2,2020-04-02,19115,PL,26.912006,32.943050,66.787497,70.071922,71.644023,72.057026
3,2020-04-15,788948,MD,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2020-03-26,158583,DE,0.349793,0.349793,0.349793,0.349793,0.349793,0.349793


#### Разделим наши данные на трейн и тест

#### Заново прочитаем наши данные чтобы лучше разбить на таблицы

In [25]:

# Шаг 1: Прочитаем данные из SQLite
conn = sqlite3.connect('data/testcase.db')

# Прочитаем таблицы из базы данных
source_comparison = pd.read_sql_query("SELECT * FROM source_comparison", conn)
costs = pd.read_sql_query("SELECT * FROM costs", conn)
revenue = pd.read_sql_query("SELECT * FROM revenue", conn)

# Закроем соединение
conn.close()

# Шаг 2: Объединим таблицы
# Объединение по столбцам 'Install_Dates' и 'Country'
merged_df = source_comparison.merge(costs, on=['Install_Dates', 'Country'], how='inner')
merged_df = merged_df.merge(revenue, on=['Install_Dates', 'Country'], how='inner')

# Шаг 3: Создание новых признаков
# Создание признаков на основе даты установки
merged_df['Install_Dates'] = pd.to_datetime(merged_df['Install_Dates'])
merged_df['Install_Year'] = merged_df['Install_Dates'].dt.year
merged_df['Install_Month'] = merged_df['Install_Dates'].dt.month
merged_df['Install_Day'] = merged_df['Install_Dates'].dt.day
merged_df['Day_of_Week'] = merged_df['Install_Dates'].dt.dayofweek

# Создание признаков на основе разницы во времени
merged_df['days_since_install'] = (pd.Timestamp.now() - merged_df['Install_Dates']).dt.days

# Кодирование категориальных переменных
categorical_columns = ['Country']
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_cats = encoder.fit_transform(merged_df[categorical_columns])
encoded_cats_df = pd.DataFrame(encoded_cats.toarray(), columns=encoder.get_feature_names_out(categorical_columns))


# Объединение закодированных признаков с основным DataFrame
merged_df = merged_df.drop(columns=categorical_columns)  # Удаляем оригинальные столбцы
merged_df = pd.concat([merged_df.reset_index(drop=True), encoded_cats_df], axis=1)

# Удаление лишних колонок
columns_to_drop = ['Install_Dates', 'source_type']
merged_df = merged_df.drop(columns=columns_to_drop)

# Проверка на пропуски и заполнение их средними значениями
merged_df.fillna(merged_df.mean(), inplace=True)

# Шаг 4: Подготовка данных для обучения модели
# Выделение целевого признака и признаков для обучения
X = merged_df.drop(['1d_LTV', '3d_LTV', '7d_LTV', '14d_LTV', '30d_LTV', '60d_LTV'], axis=1)
y = (merged_df['1d_LTV'] == 0).astype(int)  # Целевой признак - отток через 1 день

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Шаг 5: Нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Шаг 6: Обучение модели Линейной регрессии
model = LogisticRegression(random_state=42)
model.fit(X_train_scaled, y_train)

# Шаг 7: Предсказания на тренировочной и тестовой выборках
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# Шаг 8: Оценка модели
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_accuracy = roc_auc_score(y_train, y_train_pred)

test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_accuracy = roc_auc_score(y_test, y_test_pred)

print("=== Метрики для обучающей выборки ===")
print(f"Точность (Precision): {train_precision:.2f}")
print(f"Точность (roc_auc): {train_accuracy:.2f}")

print("\n=== Метрики для тестовой выборки ===")
print(f"Точность (Precision): {test_precision:.2f}")
print(f"Точность (roc_auc): {test_accuracy:.2f}")

print("\n=== Отчет классификации для тестовой выборки ===")


# Шаг 9: Пример использования модели для предсказания
example_data = X_test_scaled[0:1]  # Возьмем пример из тестовой выборки
prediction = model.predict(example_data)
print(f"Прогноз: {'Уйдет' if prediction[0] == 1 else 'Останется'}")


=== Метрики для обучающей выборки ===
Точность (Precision): 0.76
Точность (Accuracy): 0.50

=== Метрики для тестовой выборки ===
Точность (Precision): 0.60
Точность (Accuracy): 0.50

=== Отчет классификации для тестовой выборки ===
Прогноз: Останется


#### Вывод: Клиент останется 